In [785]:
#####Hierarchical bayes individual RFM model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import itertools
import math
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [786]:
####任意の相関行列(分散共分散行列)を作成する関数####
##任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = (upper - lower) *rand(col*col) + lower   #相関係数の乱数ベクトルを作成
    rho = np.reshape(np.array(cov_vec), (col, col)) * np.tri(col)   #乱数ベクトルを下三角行列化
    Sigma = np.diag(np.diag(rho + rho.T) + 1) - (rho + rho.T)   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = scipy.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(eigen_val.shape[0]-1):
        if eigen_val[i] < 0:
            eigen_val[i] = (eigen_upper - eigen_lower) * rand(1) + eigen_lower
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(pow(np.diag(Sigma), 0.5)[:, np.newaxis], pow(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

##相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

##分散共分散行列から相関行列に変換する関数
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [787]:
####データの発生####
##データの設定
k = 3
hh = 10000   #ユーザー数
period = 200   #観測期間

In [788]:
##階層モデルの説明変数を生成
#ユーザーの説明変数を生成
k1 = 3; k2 = 5; k3 = 5
u1 = np.array(np.random.random(hh*k1)).reshape((hh, k1))
u2 = np.zeros((hh, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    u2[:, j] = np.random.binomial(1, prob, hh)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hh)
u3 = np.delete(u3, np.argmin(np.sum(u3, axis=0)), axis=1)   #冗長な変数の削除
u = np.concatenate((np.repeat(1, hh)[:, np.newaxis], u1, u2, u3), axis=1)
u_col = u.shape[1]

In [789]:
##ユーザーごとにデータを生成
rp = 0
while True:
    rp = rp + 1

    #分散共分散行列を設定
    Cor = np.identity((k))
    Cor[1, 0] = Cor[0, 1] = 0.5; Cor[2, 1] = Cor[1, 2] = 0.4; Cor[0, 2] = Cor[2, 0] = 0
    tau = np.array([0.1, 0.15, 0.05])
    Cov = covmatrix(Cor, np.sqrt(tau), np.sqrt(tau))
    Covt = Cov.copy()

    #階層モデルの回帰パラメータ
    beta1 = np.array([1.75, 4.25, 1.5])
    beta2 = np.hstack((np.random.uniform(-0.55, 0.75, (k-1)*(u_col-1)).reshape(u_col-1, k-1), 
                       np.random.uniform(-0.5, 0.5, u_col-1).reshape(u_col-1, 1)))
    beta = np.vstack((beta1, beta2))
    betat = beta.copy()

    #階層モデルからモデルパラメータを生成
    theta = np.dot(u, beta) + np.random.multivariate_normal(np.repeat(0, k), Cov, hh)
    Sigma = 0.75
    thetat = theta.copy(); Sigmat = Sigma

    ##購買履歴と購買金額を生成
    #指数分布から観測期間中の購買履歴を生成
    N = 10000
    x = np.zeros(hh, dtype="int"); w = np.zeros(hh); Z = np.zeros(hh, dtype="int")
    y_list = [i for i in range(hh)]
    s_list = [i for i in range(hh)]
    id_list = [i for i in range(hh)]

    #応答変数を生成
    for i in range(hh):
        while True:
            Lambda = np.random.exponential(np.exp(theta[i, 0]), N)
            y1 = Lambda[np.cumsum(Lambda) <= period]
            y2 = np.cumsum(y1)
            if y1.shape[0] > 0:   #少なくとも1度は購買
                break

        #指数分布から離脱時間を生成
        w[i] = np.random.exponential(np.exp(theta[i, 1]), 1)
        if w[i] > period:
            w[i] = period
            Z[i] = 1

        #離脱時間から購買頻度を確定
        if np.sum(np.cumsum(y1) < w[i]) > 0:
            #再訪問ありの場合
            index = np.array(np.where(np.cumsum(y1) < w[i])[0], dtype="int32")
            y_list[i] = np.hstack((y1[index].reshape(index.shape[0], 1), y2[index].reshape(index.shape[0], 1)))
            x[i] = int(y_list[i].shape[0])
        else :    
            #再訪問なしの場合
            y_list[i] = np.zeros((1, 2))
            x[i] = int(0)

        #idを設定
        id_list[i] = np.repeat(i, y_list[i].shape[0])

        #購買金額を生成
        if x[i] > 0:
            s_list[i] = np.exp(np.random.normal(theta[i, 2], Sigma, x[i]))
        else:
            s_list[i] = np.array([0])

    #リストを変換
    user_id = np.array(list(itertools.chain(*[id_list[i] for i in range(hh)])))
    y = np.array(list(itertools.chain(*[y_list[i] for i in range(hh)])))
    s = np.array(list(itertools.chain(*[s_list[i] for i in range(hh)])))

    #break条件
    print(np.array([rp, np.sum(Z)], dtype="int32"))
    if (np.sum(Z) > hh/10) & (np.sum(Z) < hh/5) & (np.max(s) < 500):
        break

[   1 2250]
[   2 1692]
[   3 1729]


In [790]:
#インデックスを作成
y_last = np.zeros(hh)
user_list = [i for i in range(hh)]
for i in range(hh):
    user_list[i] = np.array(np.where(user_id==i)[0], dtype="int32")
    y_last[i] = y[np.max(user_list[i]), 1]   #最終購買時期

In [791]:
####マルコフ連鎖モンテカルロ法でHierarchical bayes individual RFM modelを推定####
##多変量正規分布の条件付き期待値と分散を計算する関数
def cdMVN(mu, Cov, department, U):

    #分散共分散行列のブロック行列を定義
    index = np.delete(np.arange(Cov.shape[0]), department)
    Cov11 = Cov[department, ][:, department]
    Cov12 = Cov[department, ][:, index]
    Cov21 = Cov[:, department][index, ]
    Cov22 = Cov[index, ][:, index]

    #条件付き分散と条件付き平均を計算
    CDinv = np.dot(Cov12, np.linalg.inv(Cov22))
    CDmu = mu[:, department] + np.dot(CDinv, (U[:, index] - mu[:, index]).T).T   #条件付き平均
    CDvar = Cov11 - np.dot(np.dot(Cov12, np.linalg.inv(Cov22)), Cov21)   #条件付き分散
    return CDmu, CDvar, Cov11, Cov12, Cov21, Cov22

In [792]:
##観測終了時に生存している確率
def survival_prob(theta, period, y_last):
    
    #パラメータの設定
    Lambda = 1 / np.exp(theta[:, 0])
    gamma = 1 / np.exp(theta[:, 1])

    #ユーザーごとの生存確率を計算
    weights = gamma / (Lambda + gamma)
    denom = np.exp((Lambda+gamma) * (period-y_last)) - 1
    denom[np.inf==denom] = np.power(10, 200)   #無限になった場合の対応
    Prob = 1 / (1 + weights*denom)
    return Prob

In [793]:
##切断指数分布の乱数を生成する関数
def rtexp(theta, a, b):
    
    #パラメータの設定
    gamma = np.exp(theta)

    #切断指数分布の乱数を生成
    FA = scipy.stats.expon.cdf(a, scale=gamma)
    FB = scipy.stats.expon.cdf(b, scale=gamma)
    par = scipy.stats.expon.ppf(np.random.uniform(0, 1, a.shape[0])*(FB-FA)+FA, scale=gamma)
    return par

In [794]:
##頻度と離脱の同時分布の対数尤度関数
def loglike(theta, Z, w, x_lgamma, x, y_log, y_last):

    #パラメータを設定
    Lambda = 1 / np.exp(theta[:, 0])
    gamma = 1 / np.exp(theta[:, 1])

    #頻度と離脱の対数尤度
    LLi1 = x*np.log(Lambda) + (x-1)*y_log - y_last*Lambda - x_lgamma -Lambda * (w-y_last)
    LLi2 = Z*(-gamma*w) + (1-Z)*(np.log(gamma) -gamma*w)
    LLi = LLi1 + LLi2
    return LLi

In [795]:
##多変量回帰モデルをギブスサンプリングする関数
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [796]:
##アルゴリズムの設定
LL1 = -1000000000   #対数尤度の初期値
R = 5000
keep = 5
burnin = int(R/keep)
disp = 100

In [797]:
##データの設定
#パラメータ推定用のデータの設定
N = y.shape[0]
index_k = np.arange(k-1)
censor = 1 - Z   #生存の指示変数

In [798]:
#購買頻度と離脱のデータの設定
index_repeat = np.array(np.where(x > 0)[0], dtype="int")   #リピートのインデックス
x_lgamma = scipy.special.gammaln(x); x_lgamma[np.inf==np.abs(x_lgamma)] = 0
y_log = np.log(y_last); y_log[np.inf==np.abs(y_log)] = 0
s_log = np.log(s); s_log[np.inf==np.abs(s_log)] = 0

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


In [799]:
#購買金額のデータの設定
s_log0 = s_log[s > 0]
user_id0 = user_id[s > 0]
n = np.zeros(hh)
s_mu = np.zeros(hh)
for i in range(hh):
    n[i] = s_log[user_list[i]][s[user_list[i]] > 0].shape[0]
    s_mu[i] = np.mean(s_log[user_list[i]][s_log[user_list[i]] > 0])
s_mu[np.isnan(s_mu)] = 0
s_id = np.unique(user_id[s > 0])

C:\Users\sana\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\sana\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [800]:
##事前分布の設定
#逆ガンマ分布の事前分布
s0 = 1
v0 = 1

#階層モデルの事前分布
Deltabar = np.zeros((u_col, k))
ADelta = np.diag(np.repeat(0.01, u_col))
nu = k + 1
V = nu * np.diag(np.repeat(1, k))

In [801]:
##真値を設定
#潜在変数の真値
Zi = Z.copy()
breakaway = w

#モデルパラメータの真値
theta = thetat.copy()
Sigma = Sigmat
beta = betat.copy()
Cov = Covt.copy()
beta_mu = np.dot(u, beta)

In [802]:
##初期値の設定
#モデルパラメータの初期値
theta = np.zeros((hh, k))
theta[:, 0] = 1/x; theta[np.inf==1/x, 0] = 1; theta[:, 0] = np.log(1/theta[:, 0])
theta[:, 1] = 1/y_last; theta[np.inf==1/y_last, 1] = 1/30; theta[:, 1] = np.log(1/theta[:, 1])
theta[:, 2] = np.log(s_mu); theta[np.inf==np.abs(theta[:, 2]), 2] = 0
Sigma = np.std(s_log[s > 0] )

#階層モデルの初期値
beta = np.dot(np.dot(np.linalg.inv(np.dot(u.T, u)), u.T), theta)
beta_mu = np.dot(u, beta)
Cov = np.cov(theta - beta_mu, rowvar=0)

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [803]:
#多変量回帰モデルの設定
XX = np.dot(u.T, u)
inv_XXV = np.linalg.inv(np.dot(u.T, u) + ADelta)

In [804]:
##パラメータの保存用配列
W = np.zeros((int(R/keep), hh))
B = np.zeros((int(R/keep), hh))
THETA = np.zeros((hh, k, int(R/keep)))
SIGMA = np.zeros(int(R/keep))
BETA = np.zeros((u_col, k, int(R/keep)))
COV = np.zeros((k, k, int(R/keep)))

In [805]:
##対数尤度の基準値
#初期値での対数尤度
LLst = np.sum(loglike(theta, Zi, w, x_lgamma, x, y_log, y_last))

#真値での対数尤度
LLbest = np.sum(loglike(thetat, Z, w, x_lgamma, x, y_log, y_last))

In [806]:
####MCMCでパラメータをサンプリング####
for rp in range(R):
    
    ##潜在変数および潜在離脱変数をサンプリング
    #ベルヌーイ分布から潜在生存変数をサンプリング
    Prob = survival_prob(theta, period, y_last)   #生存確率を設定
    Zi = np.random.binomial(1, Prob, hh)
    index_z1 = np.array(np.where(Zi==1)[0], dtype="int")
    index_z0 = np.delete(np.arange(hh), index_z1)

    #切断指数分布から離脱時間をサンプリング
    breakaway = np.zeros(hh)
    breakaway[index_z0] = rtexp(theta[index_z0, 1], y_last[index_z0], period)
    breakaway[index_z1] = period


    ##メトロポリスヘイスティング法で頻度と離脱のパラメータをサンプリング
    #MH法の新しいパラメータをサンプリング
    thetad = theta.copy()
    thetan = theta.copy()
    thetan[:, index_k] = thetad[:, index_k] + np.random.multivariate_normal(np.zeros(k-1), np.diag(np.repeat(0.075, k-1)), hh)

    #多変量正規分布の条件付き分布から事前分布を設定
    MVRN = cdMVN(beta_mu, Cov, index_k, thetan)
    MVRD = cdMVN(beta_mu, Cov, index_k, thetad)
    inv_Covn = np.linalg.inv(MVRN[1])
    inv_Covd = np.linalg.inv(MVRD[1])
    er_n = thetan[:, index_k] - MVRN[0]
    er_d = thetad[:, index_k] - MVRD[0]

    #対数事後分布を設定
    lognew = loglike(thetan, Zi, breakaway, x_lgamma, x, y_log, y_last)
    logold = loglike(thetad, Zi, breakaway, x_lgamma, x, y_log, y_last)
    logpnew = -1/2 * np.sum(np.dot(er_n, inv_Covn) * er_n, axis=1)
    logpold = -1/2 * np.sum(np.dot(er_d, inv_Covd) * er_d, axis=1)

    #MH法によりパラメータの採択を決定
    rand = np.random.rand(hh)
    alpha = np.min(np.hstack((np.ones((hh, 1)), np.exp(lognew + logpnew - logold - logpold).reshape(hh, 1))), axis=1)

    #alphaの値に基づき新しいthetaを採択
    flag = np.array(alpha > rand, dtype="int").reshape(hh, 1)
    theta[:, index_k] = flag*thetan[:, index_k] + (1-flag)*thetad[:, index_k]


    ##ギブスサンプリングで購買金額のパラメータをサンプリング
    #多変量正規分布の条件付き分布から事前分布を設定
    MVR = cdMVN(beta_mu, Cov, np.array([k-1], dtype="int"), theta)
    MVR_U = MVR[0].reshape(-1)
    MVR_S = np.sqrt(MVR[1])

    #正規分布から購買金額の事後分布をサンプリング
    weights = np.zeros(hh)
    weights[index_repeat] = np.power(MVR_S, 2) / (np.power(Sigma, 2)/n[index_repeat] + np.power(MVR_S, 2))
    mu_par = (weights*s_mu + (1-weights)*beta_mu[:, k-1])[index_repeat]
    theta[index_repeat, k-1] = np.random.normal(mu_par, weights[index_repeat]*np.power(Sigma, 2)/n[index_repeat], index_repeat.shape[0])

    
    #逆ガンマ分布から標準偏差をサンプリング
    er = s_log0 - theta[user_id0, k-1]   #誤差を設定
    gamma_s =  np.dot((er-np.mean(er)), (er-np.mean(er))) + s0  
    gamma_v = np.sum(s > 0) + v0
    Sigma = np.sqrt(1/np.random.gamma(gamma_v/2, 1/(gamma_s/2), 1))


    ##多変量回帰モデルから階層モデルをサンプリング
    Xy = np.dot(u.T, theta)
    out = rmultireg(theta, u, inv_XXV, Xy, Cov, ADelta, Deltabar, V, nu, hh, u_col, k)
    beta = out[0]
    beta_mu = np.dot(u, beta)
    Cov = out[1]
    inv_Cov = np.linalg.inv(Cov)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        W[mkeep, ] = Zi
        B[mkeep, ] = breakaway
        THETA[:, :, mkeep] = theta
        SIGMA[mkeep] = Sigma
        BETA[:, :, mkeep] = beta
        COV[:, :, mkeep] = Cov

    if rp%disp==0:
        #対数尤度の更新
        LL = np.sum(lognew)

        #サンプリング結果を確認
        print(rp)
        print(np.mean(alpha))
        print(np.round(np.array([LL, LLst, LLbest]), 1))
        print(np.round(np.append(Sigma, Sigmat), 3))
        print(np.round(np.hstack((cov2cor(Cov), cov2cor(Covt))), 2))

0
0.5919706268857118
[-590627.2 -272404.4  -93384.9]
[0.777 0.75 ]
[[1.   0.79 0.83 1.   0.5  0.  ]
 [0.79 1.   0.57 0.5  1.   0.4 ]
 [0.83 0.57 1.   0.   0.4  1.  ]]
100
0.6342432828892589
[-138695.4 -272404.4  -93384.9]
[0.742 0.75 ]
[[ 1.   -0.24 -0.5   1.    0.5   0.  ]
 [-0.24  1.    0.47  0.5   1.    0.4 ]
 [-0.5   0.47  1.    0.    0.4   1.  ]]
200
0.634339458145662
[-140438.4 -272404.4  -93384.9]
[0.749 0.75 ]
[[ 1.   -0.5  -0.67  1.    0.5   0.  ]
 [-0.5   1.    0.59  0.5   1.    0.4 ]
 [-0.67  0.59  1.    0.    0.4   1.  ]]
300
0.6270102486428101
[-141072.3 -272404.4  -93384.9]
[0.748 0.75 ]
[[ 1.   -0.59 -0.72  1.    0.5   0.  ]
 [-0.59  1.    0.64  0.5   1.    0.4 ]
 [-0.72  0.64  1.    0.    0.4   1.  ]]
400
0.6259980178845771
[-140256.3 -272404.4  -93384.9]
[0.749 0.75 ]
[[ 1.   -0.59 -0.72  1.    0.5   0.  ]
 [-0.59  1.    0.65  0.5   1.    0.4 ]
 [-0.72  0.65  1.    0.    0.4   1.  ]]
500
0.6251431930456529
[-140865.8 -272404.4  -93384.9]
[0.752 0.75 ]
[[ 1.   -0.62 -0.

4400
0.6270526022393093
[-141443.5 -272404.4  -93384.9]
[0.751 0.75 ]
[[ 1.   -0.61 -0.72  1.    0.5   0.  ]
 [-0.61  1.    0.68  0.5   1.    0.4 ]
 [-0.72  0.68  1.    0.    0.4   1.  ]]
4500
0.6243886347050351
[-140559.6 -272404.4  -93384.9]
[0.75 0.75]
[[ 1.   -0.6  -0.72  1.    0.5   0.  ]
 [-0.6   1.    0.67  0.5   1.    0.4 ]
 [-0.72  0.67  1.    0.    0.4   1.  ]]
4600
0.6268591960996944
[-141594.6 -272404.4  -93384.9]
[0.748 0.75 ]
[[ 1.   -0.6  -0.72  1.    0.5   0.  ]
 [-0.6   1.    0.68  0.5   1.    0.4 ]
 [-0.72  0.68  1.    0.    0.4   1.  ]]
4700
0.6265596605900812
[-140490.1 -272404.4  -93384.9]
[0.749 0.75 ]
[[ 1.   -0.64 -0.74  1.    0.5   0.  ]
 [-0.64  1.    0.69  0.5   1.    0.4 ]
 [-0.74  0.69  1.    0.    0.4   1.  ]]
4800
0.6281437683854493
[-141504.  -272404.4  -93384.9]
[0.752 0.75 ]
[[ 1.   -0.62 -0.75  1.    0.5   0.  ]
 [-0.62  1.    0.67  0.5   1.    0.4 ]
 [-0.75  0.67  1.    0.    0.4   1.  ]]
4900
0.6226288832587293
[-140325.5 -272404.4  -93384.9]
[0.75 